In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches, get_test_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '3', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/bags/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/topic_vae', 'directory of model')
flags.DEFINE_string('modelname', 'bags', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 1000, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 5, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 20, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_test_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

# build language model 

In [8]:
def debug_shape(variables):
    sample_batch = dev_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    

# run model 

## init

In [9]:
def get_feed_dict(batch, mode='train'):
    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    keep_prob = config.keep_prob if mode == 'train' else 1.0
    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow])
t_variables['keep_prob'] = tf.placeholder(tf.float32)

## build model

In [11]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors

    prob_topic = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic')(latents_bow) # inference of topic probabilities

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    topic_embeddings = tf.get_variable('topic_emb', [config.n_topic, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics

    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

## define loss

In [12]:
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(config.n_topic)))

global_step = tf.Variable(0, name='global_step',trainable=False)

loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)

grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]
opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, tf.maximum(1e-5, n_bow))
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

In [13]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch \
            = sess.run([loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_reg_mean, ppl_mean

def print_topic_sample():
    topics_freq_bow_idxs = bow_idxs[sess.run(topics_freq_bow_indices)]
    for i, topic_freq_bow_idxs in enumerate(topics_freq_bow_idxs):
        print(i, ' BOW:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))

In [14]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','TM','','','','VALID:','TM','','',''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG']]))))

In [15]:
if len(log_df) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, topic_ppls], feed_dict = feed_dict)
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev = get_loss(sess, dev_batches)
            global_step_log = sess.run(tf.train.get_global_step())
            
#             if loss_dev < loss_min:
#                 loss_min = loss_dev
#                 saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()
    
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            
            # visualize topic
            print_topic_sample()

            time_start = time.time()
            
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

TRAIN:    TM                      VALID:    TM          \
       Time  Ep    Ct    LOSS   PPL     NLL    KL   REG    LOSS   PPL     NLL   
1         0   0     0  126.37  1035  124.96  0.47  0.95  117.38  1034  116.13   
1001      4   0  1000  113.89   567  113.15  0.30  0.44  105.48   522  104.90   
2001      4   0  2000  113.04   541  112.19  0.54  0.32  104.88   500  104.00   
2276      1   1     0  112.99   536  112.10  0.59  0.30  104.51   485  103.62   
3276      4   1  1000  112.48   520  111.46  0.78  0.24  104.09   468  102.89   
4276      4   1  2000  112.22   506  111.08  0.94  0.20  103.70   445  102.21   
4551      1   2     0  112.13   503  110.95  0.98  0.19  103.65   445  102.09   
5551      4   2  1000  111.78   492  110.50  1.11  0.17  103.36   438  101.75   
6551      4   2  2000  111.60   483  110.24  1.21  0.15  103.40   434  101.76   
6826      1   3     0  111.58   481  110.20  1.24  0.14  103.39   433  101.69   
7826      4   3  1000  111.35   474  109.90  1.33  0.13  103.14   423  101.32   
8826      4   3  2000  111.22   468  109.71  1.40  0.12  103.18   421  101.26   
9101      1   4     0  111.21   467  109.67  1.42  0.11  103.23   424  101.33   
10101     4   4  1000  111.11   462  109.52  1.49  0.11  103.25   422  101.29   
11101     4   4  2000  110.96   458  109.31  1.55  0.10  103.00   412  101.00   
11376     1   5     0  110.93   456  109.28  1.56  0.10  102.97   414  101.01   
12376     4   5  1000  110.85   452  109.15  1.61  0.09  102.93   414  100.90   
13376     4   5  2000  110.74   449  108.99  1.66  0.09  103.00   410  100.87   
13651     1   6     0  110.73   448  108.97  1.67  0.08  102.94   413  100.88   
14651     4   6  1000  110.64   445  108.84  1.72  0.08  102.91   409  100.80   
15651     4   6  2000  110.58   442  108.75  1.75  0.08  103.10   413  100.97   
15926     1   7     0  110.56   442  108.72  1.76  0.07  102.76   402  100.58   
16926     4   7  1000  110.49   439  108.62  1.80  0.07  102.75   401  100.53   
17926     4   7  2000  110.45   437  108.54  1.84  0.07  102.86   402  100.63   
18201     1   8     0  110.42   436  108.51  1.85  0.07  102.75   402  100.52   
19201     4   8  1000  110.39   434  108.44  1.88  0.07  102.76   404  100.54   
20201     4   8  2000  110.32   432  108.35  1.91  0.06  102.84   404  100.53   
20476     1   9     0  110.31   432  108.33  1.91  0.06  102.76   398  100.47   
21476     4   9  1000  110.27   430  108.27  1.94  0.06  102.69   402  100.42   
22476     4   9  2000  110.23   428  108.20  1.97  0.06  102.72   397  100.36   
...     ...  ..   ...     ...   ...     ...   ...   ...     ...   ...     ...   
91001     1  40     0  109.29   392  106.70  2.56  0.03  102.32   382   99.64   
92001     4  40  1000  109.29   392  106.69  2.57  0.03  102.44   383   99.73   
93001     4  40  2000  109.28   391  106.68  2.57  0.03  102.30   385   99.67   
93276     1  41     0  109.28   391  106.68  2.57  0.03  102.17   377   99.50   
94276     4  41  1000  109.27   391  106.67  2.57  0.03  102.28   382   99.62   
95276     4  41  2000  109.27   391  106.66  2.58  0.03  102.43   385   99.81   
95551     1  42     0  109.27   391  106.66  2.58  0.03  102.39   384   99.70   
96551     4  42  1000  109.27   391  106.65  2.58  0.03  102.36   384   99.72   
97551     4  42  2000  109.26   391  106.64  2.59  0.03  102.27   381   99.65   
97826     1  43     0  109.26   391  106.64  2.59  0.03  102.31   381   99.68   
98826     4  43  1000  109.25   390  106.63  2.59  0.03  102.30   381   99.65   
99826     4  43  2000  109.25   390  106.62  2.59  0.03  102.49   385   99.82   
100101    1  44     0  109.25   390  106.62  2.60  0.03  102.37   384   99.67   
101101    4  44  1000  109.24   390  106.61  2.60  0.03  102.30   378   99.63   
102101    4  44  2000  109.24   390  106.60  2.60  0.03  102.35   384   99.71   
102376    1  45     0  109.24   390  106.60  2.60  0.03  102.34   381   99.70   
103376    4  45  1000  109.23   389  106.59  2.61  0.0

0  BOW: ... 'm cheap pretty thing price nice time bad put
1  BOW: pocket room power mouse ipad carry charger small perfect cord
2  BOW: carry room camera pockets stuff work books plenty comfortable space
3  BOW: zipper - : padding inside quality flap side velcro material
4  BOW: pro retina speck apple shell scratches display mbp finish hard
5  BOW: ; & pro air big perfectly size nice ... work
6  BOW: ! love color perfect buy recommend absolutely perfectly ... compliments
7  BOW: pockets strap compartment shoulder pocket straps side small front handle
8  BOW: netbook acer aspire mini asus toshiba battery samsung wireless pc
9  BOW: item received return ordered amazon arrived back disappointed money seller
10  BOW: bottom top cover plastic part back feet piece corners rubber
11  BOW: plain lose chemical odor smells print sewn smell sd swiss
12  BOW: inch laptops sleeve inches size dimensions asus hp tight larger
13  BOW: sleeve protection air protect inside pro snug neoprene nice perfect

# confirm variables

In [16]:
debug_value([topic_losses_recon, n_bow])

Neg:0 : [ 62.297527  56.136356 154.29373   88.97808  102.96145   87.09083
 227.5929    97.16991   73.68268   71.77301   31.193447 142.24402
  72.797424  62.079376  71.74945  145.35117   87.5488   117.33721
  27.166435  64.628746  44.766636  56.912777  66.08688   85.55026
 105.20309  104.787796  80.63867   57.57313   91.79883   82.65097
  84.80786   67.81983   29.320303 176.21875   38.674168 275.64484
  46.82713  171.4018    71.25316   69.34422   54.380184  85.18074
 102.59453   85.81177  126.17367   93.92539  120.23218   98.609024
  70.81109  238.25818  119.59937  179.05786  139.86867 ]
Sum_2:0 : [10. 10. 26. 14. 15. 14. 39. 17. 13. 12.  5. 24. 13. 11. 12. 23. 14. 19.
  5. 12.  7.  9. 11. 15. 16. 19. 14.  9. 16. 14. 14. 12.  5. 29.  7. 44.
  9. 28. 12. 12. 10. 14. 17. 14. 21. 17. 19. 17. 12. 38. 19. 27. 25.]


In [17]:
debug_value([tf.exp(-tf.divide(topic_losses_recon, n_bow))])

Exp_1:0 : [0.00177701 0.00393379 0.00284705 0.00204366 0.00117886 0.00261166
 0.00293802 0.00326275 0.00467356 0.00429009 0.00235161 0.00262558
 0.00342418 0.00388673 0.00315612 0.0017324  0.00197161 0.00368242
 0.0028409  0.00355141 0.00157192 0.001876   0.00237097 0.0025489
 0.00120646 0.00365071 0.00304019 0.00170096 0.00249563 0.00308826
 0.0036157  0.0036778  0.00452899 0.0020654  0.00542014 0.00180567
 0.00704606 0.00215958 0.00298581 0.00290098 0.00407997 0.00248433
 0.00203083 0.00226861 0.00196002 0.00388822 0.00168553 0.0027972
 0.00260488 0.00200653 0.00182105 0.00125839 0.00407746]


### test

In [18]:
debug_shape([bow, hidden_bow, latents_bow, prob_topic, bow_embeddings, topic_embeddings, topic_bow, prob_bow])

NameError: name 'bow' is not defined

In [ ]:
debug_shape([topic_losses_recon, topic_loss_recon, n_bow, ppls, topic_embeddings_norm, tf.expand_dims(topic_angles_mean, -1), topic_angles_vars])

In [ ]:
debug_value([tf.reduce_sum(tf.square(topic_embeddings_norm), 1)], return_value=True)[0]

In [ ]:
debug_value([tf.reduce_sum(prob_topic, -1), tf.reduce_sum(topic_bow, -1), tf.reduce_sum(tf.exp(prob_bow), 1)])

In [ ]:
sigma_bow = tf.exp(0.5 * logvars_bow)
dist_bow = tfd.Normal(means_bow, sigma_bow)
dist_std = tfd.Normal(0., 1.)
topic_loss_kl_tmp = tf.reduce_mean(tf.reduce_sum(tfd.kl_divergence(dist_bow, dist_std), 1))

In [ ]:
debug_value([topic_loss_recon, topic_loss_kl, topic_loss_kl_tmp])

In [ ]:
_logvars, _means, _kl_losses, _latents, _output_logits = sess.run([logvars, means, kl_losses, latents, output_logits], feed_dict=feed_dict)


In [ ]:
_logvars.shape, _means.shape, _kl_losses.shape, _latents.shape

In [ ]:
_output_logits

In [ ]:
_output_logits, _dec_target_idxs_do, _dec_mask_tokens_do, _recon_loss, _kl_losses, _ = sess.run([output_logits, dec_target_idxs_do, dec_mask_tokens_do, recon_loss, kl_losses, opt], feed_dict=feed_dict)


In [ ]:
tf.reduce_max(output_logits, 2).eval(session=sess, feed_dict=feed_dict).shape

In [ ]:
_output_logits.shape, _dec_target_idxs_do.shape, _dec_mask_tokens_do.shape

In [ ]:
_logits = np.exp(_output_logits) / np.sum(np.exp(_output_logits), 2)[:, :, None]

In [ ]:
_idxs = _dec_target_idxs_do

In [ ]:
_losses = np.array([[-np.log(_logits[i, j, _idxs[i, j]]) for j in range(_idxs.shape[1])] for i in range(_idxs.shape[0])]) * _dec_mask_tokens_do

In [ ]:
np.sum(_losses)/np.sum(_dec_mask_tokens_do)

In [ ]:
_recon_loss

In [ ]:
_kl_losses.shape